# 3장 - 함수


- 프로그래머들이 파이썬에서 처음으로 사용하는 정리 도구는 함수(function)다.

### BETTER WAY 19 - 함수가 여러 값을 반환하는 경우 절대로 네 값 이상을 언패킹하지 말라

In [2]:
# 2개를 리턴하는 함수
def get_stats(numbers):
    mininum = min(numbers)
    maximum = max(numbers)
    return mininum, maximum

lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

minimum, maximum = get_stats(lengths)
print(f'최소: {minimum}, 최대: {maximum}')

최소: 60, 최대: 73


In [5]:
# 5개를 리턴하는 함수
def get_stats(numbers):
    mininum = min(numbers)
    maximum = max(numbers)
    count = len(numbers)
    average = sum(numbers) / count
    
    sorted_numbers = sorted(numbers)
    middle = count // 2
    if count % 2 == 0:
        lower = sorted_numbers[middle - 1]
        upper = sorted_numbers[middle]
        median = (lower + upper) / 2
    else:
        median = sorted_numbers[middle]
        
    return minimum, maximum, average, median, count

In [6]:
minimum, maximum, average, median, count = get_stats(lengths)
print(f'최소: {minimum}, 최대: {maximum}, 평균: {average}, 중앙값: {median}, 개수: {count}')

최소: 60, 최대: 73, 평균: 67.5, 중앙값: 68.5, 개수: 10


- 이 코드에는 문제가 있다.
- 먼저 모든 반환 값이 number 이기 때문에 순서를 혼동하기 쉽다
- 반환 값이 많으면 많을수록 실수하기도 아주 쉬워진다

- <font color='red'>이런 문제를 피하려면 함수가 여러 값을 반환하거나 언패킹할 때 값이나 변수를 네 개 이상 사용하면 안된다.</font>
- 많은 값을 언패킹해야 한다면 경량 클래스(lightweight class)나 namedtuple을 사용하고, 함수도 이런 값을 반환하게 만드는 것이 낫다.

### BETTER WAY 20 - None을 반환하기보다는 예외를 발생시켜라

- 파이썬 프로그래머들은 유틸리티 함수를 작성하면서 None 을 리턴시키면서 이 값에 특별한 의미를 부여하려는 경향을 나타낸다
- 그런데 함수가 반환한 결과를 if 문 등의 조건에서 평가할 때 0 값이 문제가 될 수 있다.
- None 을 검사하는 대신 실수로 빈 값을 취급하는 오류를 야기하기 쉽다

In [1]:
# example
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [2]:
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

잘못된 입력


In [3]:
x, y = 0, 1
result = careful_divide(x, y)
if result is None:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

device result: 0.0


In [4]:
x, y = 0, 1
result = careful_divide(x, y)
if not result: # 이렇게 처리하는 실수를 간혹 할 수 있다. 아니 이렇게가 흔히 저지르는 실수다.
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

잘못된 입력


- 이런 실수 할 가능성을 줄이는 방법은 2가지

In [8]:
'''
방법 1 - 반환 값을 2-튜플로 분리
'''
def careful_divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

x, y = 1, 0
success, result = careful_divide(x, y)
if not success:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')
    
x, y = 0, 1
success, result = careful_divide(x, y)
if not success:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')
    
x, y = 0, 1
_, result = careful_divide(x, y)
if not result:                       # 이런 오류의 발생 가능성이 여전히 남아있다.
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

잘못된 입력
device result: 0.0
잘못된 입력


In [10]:
'''
방법 2 - 결코 None 을 반환하지 않는 것. 대신 Exception 을 호출한 쪽으로 발생시켜서 호출차가 처리하도록 하는 것.
'''
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')

x, y = 1, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')
    
x, y = 0, 1
try:
    result = careful_divide(x, y)
except ValueError:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

잘못된 입력
device result: 0.0


In [11]:
# 독스트링(Docstring)과 타입 애너테이션까지 포함시키는 코드
def careful_divide(a, b) -> float:
    """
    a 를 b 로 나눈다.
    
    Raises:
        ValueError: b 가 0이어서 나눗셈을 할 수 없을 때
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('잘못된 입력')
        
x, y = 1, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print(f'잘못된 입력')
else:
    print(f'device result: {result}')

잘못된 입력


### BETTER WAY 21 - 변수 영역과 클로저의 상호작용 방식을 이해하라

- 숫자로 이뤄진 리스트를 정렬하되, 정렬한 리스트의 앞쪽에는 우선순위를 부여한 몇 몇 숫자를 위치시켜야 한다고 가정하자

- 이를 해결하는 일반적인 방법은 sort 메서드에 key 인자로 도우미 함수를 전달하는 것

In [15]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [18]:
'''
이게 예상대로 작동하는 세 가지 이유
1. 파이썬이 클로저(closure)를 지원
  - 클로저란 자신이 정의된 영역 밖의 변수를 참조하는 함수
  - 이로 인해 helper 함수가 group 변수를 접근할 수 있음
2. 파이썬에서 함수가 일급 시민(first-class citizen) 객체임
  - 일급 시민 객체란 맡은 이를 직접 가리킬 수 있고, 변수에 대입하거나, 다른 함수에 인자로 전달할 수 있으며,
    if 문에서 함수를 비교하거나 반환하는 것 등이 가능하다는 것을 의미한다.
3. 파이썬에는 시퀀스(튜플 포함)를 비교하는 구체적인 규칙이 있음
'''

class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
print(sorter.found)

True


### BETTER WAY 22 - 변수 위치 인자를 사용해 시각적인 잡음을 줄여라

In [19]:
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')
        
log('내 숫자는', [1, 2])
log('안녕', [])           # 로그에 남길 게 없어도 불필요하게 빈 리스트를 넘겨야 한다.

내 숫자는: 1, 2
안녕


In [20]:
'''
개선 코드
'''
def log(message, *values): # 마지막 인자 앞에 * 을 붙임응
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')
        
log('내 숫자는', 1, 2)
log('안녕')                # 훨씬 좋다

내 숫자는: 1, 2
안녕


### BETTER WAY 23 - 키워드 인자로 선택적인 기능을 제공하라

In [21]:
def remainder(number, devisor):
    return number % devisor

print(remainder(20, 7))

6


- 다른 대부분의 프로그래밍 언어와 마찬가지로 파이썬에서도 함수를 호출할 때 위치에 따라 인자를 넘길 수 있다.

In [24]:
print(remainder(20, 7))
print(remainder(20, devisor=7))
print(remainder(number=20, devisor=7))
print(remainder(devisor=7, number=20))

6
6
6
6


In [25]:
# 위치 기반 인자를 지정하려면 키워드 인자보다 앞에 지정해야 한다.
# 다음과 같이 하면 에러
print(remainder(number=20, 7))

SyntaxError: positional argument follows keyword argument (<ipython-input-25-5c64d3bcdf6b>, line 3)

### BETTER WAY 24 - None과 독스트링을 사용해 동적인 디폴트 인자를 지정하라

- 종종 키워드 인자의 값으로 정적으로 정해지지 않는 타입의 값을 써야 할 때가 있다

In [26]:
# ex
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')
    
log('안녕!')
sleep(0.1)
log('다시 안녕!')

2021-01-07 20:10:45.824396: 안녕!
2021-01-07 20:10:45.824396: 다시 안녕!


- 하지만 디폴트 인자는 이런 식으로 작동하지 않는다.
- 함수가 정의되는 시점에 datetime.now 가 한 번 호출되지 때문에 타임스탬프가 항상 같다

- 이런 경우 원하는 동작을 달성하는 파이썬의 일반적인 관례는, 디폴트 값으로 None 을 지정하고, 실제 동작을 독스트링에 문서화하는 것이다.

In [28]:
def log(message, when=None):
    """메시지와 타임스탬프를 로그에 남긴다
    
    Args:
        message: 출력할 메시지
        when: 메시지가 발생한 시각 (datetime)
            디폴트 값은 현재 시간이다.    
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log('안녕!')
sleep(0.1)
log('다시 안녕!')

2021-01-07 20:23:40.593942: 안녕!
2021-01-07 20:23:40.693961: 다시 안녕!


### BETTER WAY 25 - 위치로만 인자를 지정하게 하거나 키워드로만 인자를 지정하게 해서 함수 호출을 명확하게 만들라

- 키워드를 사용해 인자를 넘기는 기능은 파이썬 함수의 강력한 기능이다.

In [31]:
# ex)
# 한 숫자를 다른 숫자로 나누는 함수를 만드는데, 
# 때론 ZeroDivisionError 예외를 무시하고 무한대를 리턴하고 싶고
# 때론 OverflowError 예외를 무시하고 0 을 리턴하고 싶은 함수를 만들고자 한다.
def safe_division(number, divisor,
                 ignore_overflow,
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# 다음 코드는 오버플로우를 무시하고 0 을 리턴
result = safe_division(1.0, 10**500, True, False)
print(result)

# 다음 코드는 제로 디비전을 무시하고 inf 를 리턴
result = safe_division(1.0, 0, False, True)
print(result)

0
inf


- 이 함수는 사용하긴 쉽지만, 어떤 예외를 무시할 지 결정하는 두 Bool 변수의 위치를 혼동하기 쉽다.
- <font color='blue'>그러므로, 이 코드의 가독성을 향상시키는 방법은 키워드 인자를 사용하는 것이다.</font>

In [32]:
# 기본적으로 모든 예외는 다시 던진다.
def safe_division(number, divisor,
                 ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
# 이제 호출하는 쪽에서 키워드 인자를 사용해서 무시할 예외를 정해야 한다.
result = safe_division(1.0, 10**500, ignore_overflow=True)
print(result)

result = safe_division(1.0, 0, ignore_zero_division=True)
print(result)

0
inf


- 하지만 호출하는 쪽에서 명확성을 위해 키워드 인자를 꼭 쓰도록 강요할 수 는 없다
- 또한, 새로 정의한 safe_devision 에서도 이전과 같은 방법으로 함수를 호출할 수 있다.

In [34]:
result = safe_division(1.0, 10**500, True, False)
print(result)

result = safe_division(1.0, 0, False, True)
print(result)

0
inf


- 따라서, 호출자가 "키워드만 사용하는 인자"를 통해 의도를 명확히 밝히도록 요구하는 편이 좋다.
- 이는 인자 목록에 * 를 추가함으로써 가능하다.

In [35]:
def safe_division(number, divisor, *,                # 변경
                 ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
result = safe_division(1.0, 10**500, ignore_overflow=True)
print(result)

result = safe_division(1.0, 0, ignore_zero_division=True)
print(result)

0
inf


In [36]:
# 키워드 인자 지정 없이 하면 프로그램이 제대로 동작하지 않는다.
result = safe_division(1.0, 10**500, True, False)
print(result)

TypeError: safe_division() takes 2 positional arguments but 4 were given

- 하지만 이 함수에도 문제가 있다.
- 맨 앞에 있는 두 필수 인자(number, divisor)를 호출하면서 위치와 키워드를 혼용할 수 도 있다.

In [38]:
result = safe_division(number=2, divisor=5)
print(result)

result = safe_division(divisor=5, number=2)
print(result)

0.4
0.4


- 또한, 나중에 스타일이 바뀌어서 맨 앞의 두 인자 이름을 변경할 수 도 있다.

In [39]:
def safe_division(nn, dd, *,                # 변경
                 ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
# 이 경우 기존에 safe_divison 함수를 호출할때 이름 인자를 사용할 것이라고 예상을 못했기 때문에 오류가 발생한다.
result = safe_division(number=2, divisor=5)
print(result)

TypeError: safe_division() got an unexpected keyword argument 'number'

- 파이썬 3.8 에서는 이 문제에 대한 해법이 들어 있다.
- 이를 <font color='blue'>"위치로만 지정하는 인자"</font> 라고 부른다.
- 위치로만 지정하는 인자는 반드시 위치만 사용해 인자를 지정해야 하고, 키워드 인자로는 쓸 수 없다.
- 이를 위해 위치로만 지정하고자 하는 인자 뒤에 / 기호를 추가함으로써, 위치로만 지정하는 인자의 끝을 표시한다.

In [40]:
def safe_division(number, divisor, /, *,                # 변경
                 ignore_overflow=False,
                 ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [41]:
# 이제 앞에 2개 인자는 반드시 위치로만 지정해야 한다.
result = safe_division(2, 5)
print(result)

0.4


In [42]:
# 위치로만 지정해야 하는 인자에 키워드로 지정하면 오류 발생한다.
result = safe_division(number=2, divisor=5)
print(result)

TypeError: safe_division() got some positional-only arguments passed as keyword arguments: 'number, divisor'

### BETTER WAY 26 - functools.wrap 을 사용해 함수 데코레이터를 정의하라

- 파이썬은 함수에 적용할 수 있는 데코레이터(decorator)를 정의하는 특별한 구문을 제공한다.
- 데코레이터는 자신이 감싸고 있는 함수가 호출되기 전과 후에 코드를 추가로 실행해준다.

In [43]:
# 예)
# 함수가 호출될 때마다 인자 값과 반환 값을 출력하고 싶다고 하자
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        return result
    return wrapper

In [48]:
# 이 데코레이터를 함수에 적용할 때는 @ 기호를 사용한다.
@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

In [49]:
# 그냥 함수 호출
fibonacci(4)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3


3

In [50]:
# @ 기호를 사용하는 것은 이 함수에 대해 데코레이터를 호출한 후, 
#   데코레이터가 반환하는 결과를 원래 함수가 속해야 하는 영역에 원래 함수와 같은 이름으로 등록하는 것과 같다.

fibonacci = trace(fibonacci) # 이렇게 꾸며진 함수(새로운 fibonacci)는 wrapper의 코드를 원래의 fibonacci함수가 실행되기 전과 후에 실행한다.

fibonacci(4) # 따라서 wrapper 는 재귀 스택의 매 단계마다 함수의 인자와 반환 값을 출력한다.

fibonacci((0,), {}) -> 0
wrapper((0,), {}) -> 0
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((2,), {}) -> 1
wrapper((2,), {}) -> 1
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((0,), {}) -> 0
wrapper((0,), {}) -> 0
fibonacci((1,), {}) -> 1
wrapper((1,), {}) -> 1
fibonacci((2,), {}) -> 1
wrapper((2,), {}) -> 1
fibonacci((3,), {}) -> 2
wrapper((3,), {}) -> 2
fibonacci((4,), {}) -> 3
wrapper((4,), {}) -> 3


3

In [53]:
# 이 코드는 잘 동작하지만, 의도하지 않은 부작용이 있다.
# 데코레이터가 반환하는 함수(앞에서 호출한 감싸진 fibonacci 함수)의 이름이 fibonacci가 아니게 된다.

print(fibonacci)

<function trace.<locals>.wrapper at 0x000002884C4B8A60>


In [54]:
# 다음과 같이 호출하면 fibonacci 함수의 맨 앞부분에 있는 독스트링이 출력돼야 하지만, 실제로는 그렇지 않다.
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [55]:
# 데코레이터가 감싸고 있는 원래 함수의 위치를 찾을 수 없기 때문에, 객체 직렬화도 깨진다.
import pickle
pickle.dumps(fibonacci)


AttributeError: Can't pickle local object 'trace.<locals>.wrapper'

- 문제를 해결하는 방법은
- <font color='blue'>functools 내장 모듈에 정의된 wraps 도우미 함수를 사용하는 것이다.</font>
- 이 함수는 데코레이터 작성을 돕는 데코레이터다.

In [65]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')
        return result
    return wrapper

@trace
def fibonacci(n):
    """n번째 피보나치 수를 반환한다."""
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

In [68]:
# help 함수를 실행하면 원하는 결과를 볼 수 있다.
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    n번째 피보나치 수를 반환한다.



In [69]:
# pickle 객체 직렬화도 제대로 작동한다
import pickle
pickle.dumps(fibonacci)

b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tfibonacci\x94\x93\x94.'